# **Transfer Learning**

## 1. What is Transfer Learning?

Transfer learning is when we **reuse a pre-trained model** (already trained on a large dataset like ImageNet) for a **new task**.  
Instead of training from scratch, we **leverage the learned features** (edges, shapes, textures) and adapt the model for a new dataset (usually smaller).

> Think of it like:  
> *"Borrowing someone else’s knowledge and adjusting it to solve your own problem."*

---

## 2. Why Use Transfer Learning?

1. **Speeds up training** – you don’t start from zero.
2. **Requires less data** – works even with smaller datasets.
3. **Achieves better accuracy** – especially for small datasets.
4. **Uses fewer computing resources.**

---

## 3. Steps in Transfer Learning

### **Step 1: Choose a Pre-trained Model**
- Examples: **VGG16, ResNet50, InceptionV3, MobileNet, EfficientNet**
- These models are trained on **ImageNet (1M+ images, 1000 classes)**.

### **Step 2: Load the Model**
- Load it with or without the top classification layer:
  - `include_top=True` → keeps the original classifier (for ImageNet).
  - `include_top=False` → removes the classifier so you can add your own.

### **Step 3: Freeze the Base Layers**
- The convolutional layers already learned general features.
- Set `trainable=False` so their weights don’t change.

### **Step 4: Add Your Own Classifier**
- Add Dense layers for your dataset:
  - Example: `Flatten → Dense(128, ReLU) → Dense(num_classes, Softmax)`

### **Step 5: Train the New Layers**
- Train only the new layers for a few epochs.
- Use a **low learning rate**.

### **Step 6: Fine-Tuning (Optional)**
- Unfreeze some of the last convolutional layers.
- Train again with an **even lower learning rate** to adjust deeper features.

---

## 4. Visual Flow (Concept)

1. Pre-trained model (e.g., ResNet, VGG)  
2. Remove the old classifier  
3. Add your custom classification head  
4. Train the new layers  
5. Optionally fine-tune deeper layers  

---

## 5. Example Code (Keras)

```python
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Step 1: Load Pre-trained Model (without top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Step 2: Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Step 3: Add custom classifier
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Suppose we have 10 classes
])

# Step 4: Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Train
model.fit(train_data, epochs=5, validation_data=val_data)

# Step 6 (Optional): Fine-tuning
for layer in base_model.layers[-4:]:  # unfreeze last 4 layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=3, validation_data=val_data)

```

# 6 When to Use Transfer Learning?

Transfer learning is most effective when:

- Your dataset is **small (less than 10,000 images)**.  
- You need **fast training** and **don’t want to train from scratch**.  
- Your task is **similar to the dataset the pre-trained model was trained on**,  
  *for example: natural images → natural images*.
